# NB QuasiCopula.jl vs. lme4, glmmTMB, GLMMadaptive

In this notebook we compare the run times and estimates from our NB VCM fit vs that of R packages `lme4`, `glmmTMB` and `GLMMadaptive`. 

In this example, the dataset was simulated under the random intercept negative Binomial GLMM for a cluster size of 5 and sample size of 10000.

We will use the `RCall` package from julia to get the following results. 

In [1]:
using QuasiCopula, LinearAlgebra, DelimitedFiles, GLM, RCall
using Random
using DataFrames

For these results we can leverage parallelization features of the Julia language to compute across 8 cores on the following machine.

In [2]:
versioninfo()

Julia Version 1.6.2
Commit 1b93d53fc4 (2021-07-14 15:36 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin18.7.0)
  CPU: Intel(R) Core(TM) i9-9880H CPU @ 2.30GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, skylake)
Environment:
  JULIA_NUM_THREADS = 8


In [3]:
BLAS.set_num_threads(1)
Threads.nthreads()

8

Here we simulate a dataset under the random intercept negative Binomial GLMM for a cluster size of 5 and sample size of 10000.

In [4]:
function __get_distribution(dist::Type{D}, μ, r) where D <: UnivariateDistribution
    return dist(r, μ)
end

p_fixed = 3    # number of fixed effects, including intercept
m = 1    # number of variance components
# true parameter values
Random.seed!(1234)
βtrue = rand(Uniform(-0.2, 0.2), p_fixed)
θtrue = [0.01]
rtrue = 10.0

# #simulation parameters
samplesize = 10000
ni = 5 # number of observations per individual

d = NegativeBinomial()
link = LogLink()
D = typeof(d)
Link = typeof(link)
T = Float64;

In [5]:
gcs = Vector{NBCopulaVCObs{T, D, Link}}(undef, samplesize)

Γ = θtrue[1] * ones(ni, ni) + 0.00000000000001 * Matrix(I, ni, ni)
a = collect(1:samplesize)
group = [repeat([a[i]], ni) for i in 1:samplesize]
groupstack = vcat(group...)
Xstack = []
Ystack = []
Random.seed!(12345)
X_samplesize = [randn(ni, p_fixed - 1) for i in 1:samplesize]
for i in 1:samplesize
    X = [ones(ni) X_samplesize[i]]
    η = X * βtrue
    V = [ones(ni, ni)]
    # generate mvn response
    mvn_d = MvNormal(η, Γ)
    mvn_η = rand(mvn_d)
    μ = GLM.linkinv.(link, mvn_η)
    p_nb = rtrue ./ (μ .+ rtrue)
    y = Float64.(rand.(__get_distribution.(D, p_nb, rtrue)))
    # add to data
    gcs[i] = NBCopulaVCObs(y, X, V, d, link)
    push!(Xstack, X)
    push!(Ystack, y)
end

# form NBCopulaVCModel
gcm = NBCopulaVCModel(gcs)

Quasi-Copula Variance Component Model
  * base distribution: NegativeBinomial
  * link function: LogLink
  * number of clusters: 10000
  * cluster size min, max: 5, 5
  * number of variance components: 1
  * number of fixed effects: 3

In [6]:
println("precompiling NB VCM fit")
gcm2 = deepcopy(gcm);
QuasiCopula.fit!(gcm2, maxBlockIter = 1); # precompile here, and run again next cell

precompiling NB VCM fit
initializing β using GLM.jl
gcm.β = [0.036773196910952585, 0.10589609669649325, 0.02630211202720162]
initializing variance components using MM-Algorithm
gcm.θ = [1.0201776662515867e-6]
initializing r using Newton update
Converging when tol ≤ 1.0e-6 (max block iter = 1)

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Block iter 1 r = 9.84, logl = -67518.72, tol = 67518.71639191419


In [7]:
# true fit time for QC
fittime = @elapsed QuasiCopula.fit!(gcm)

initializing β using GLM.jl
gcm.β = [0.036773196910952585, 0.10589609669649325, 0.02630211202720162]
initializing variance components using MM-Algorithm
gcm.θ = [1.0201776662515867e-6]
initializing r using Newton update
Converging when tol ≤ 1.0e-6 (max block iter = 10)
Block iter 1 r = 9.84, logl = -67518.72, tol = 67518.71639191419
Block iter 2 r = 9.97, logl = -67518.23, tol = 7.167998681848435e-6


1.246861772

In [8]:
@show fittime
@show gcm.β
@show gcm.θ
@show gcm.r;

fittime = 1.246861772
gcm.β = [0.03273834387319724, 0.10609650672731126, 0.026158048770722593]
gcm.θ = [0.007030480473798856]
gcm.r = [10.001819984466927]


In [9]:
get_CI(gcm)

5×2 Matrix{Float64}:
 0.0281207   0.037356
 0.100997    0.111196
 0.0172389   0.0350772
 9.09354    10.9101
 0.0033213   0.0107397

In [10]:
@time get_CI(gcm)

  0.046255 seconds (30.07 k allocations: 477.531 KiB)


5×2 Matrix{Float64}:
 0.0281207   0.037356
 0.100997    0.111196
 0.0172389   0.0350772
 9.09354    10.9101
 0.0033213   0.0107397

# Using lme4 package in R

In [11]:
Xstack = [vcat(Xstack...)][1]
Ystack = [vcat(Ystack...)][1]
# p = 3
df = DataFrame(Y = Ystack, X2 = Xstack[:, 2], X3 = Xstack[:, 3], group = string.(groupstack))

@rput df

,Y,X2,X3,group
,Float64,Float64,Float64,String
1,1.0,1.17236,0.282272,1
2,2.0,0.852707,-1.47708,1
3,1.0,0.415565,0.126747,1
4,1.0,0.516425,1.46647,1
5,1.0,0.685759,-0.158158,1
6,1.0,0.606808,-1.30436,2
7,0.0,2.00645,0.378353,2
8,0.0,-0.588508,1.21656,2
9,2.0,0.0736438,1.12871,2


In [12]:
# time the fit
R"""
    library("lme4")
    ptm <- proc.time()
    m.nb <- glmer.nb(Y ~ 1 + X2 + X3 + (1|group), data = df, verbose = TRUE)
    # Stop the clock
    proc.time() - ptm
"""

┌ Warning: RCall.jl: Warning: package ‘lme4’ was built under R version 4.0.2
│ Loading required package: Matrix
│ Warning: package ‘Matrix’ was built under R version 4.0.2
└ @ RCall /Users/sarahji/.julia/packages/RCall/6kphM/src/io.jl:172


th := est_theta(glmer(..)) = 19.64933 --> dev.= -2*logLik(.) = 135075.4 
 1: th=    9.677844626, dev=135035.72917512, beta[1]=    0.03267970
 2: th=    39.89484346, dev=135135.98326557, beta[1]=    0.02457419
 3: th=    4.032760026, dev=135363.52137544, beta[1]=    0.03677699
 4: th=    16.37695964, dev=135059.64392083, beta[1]=    0.02812043
 5: th=    6.927303790, dev=135066.52533935, beta[1]=    0.03677240
 6: th=    10.92096838, dev=135036.15976158, beta[1]=    0.03136567
 7: th=    10.10502787, dev=135035.36887220, beta[1]=    0.03218595
 8: th=    10.16185686, dev=135035.36652613, beta[1]=    0.03212515
 9: th=    10.14646636, dev=135035.36618983, beta[1]=    0.03214196
10: th=    10.14696051, dev=135035.36618935, beta[1]=    0.03214196
11: th=    10.14712998, dev=135035.36618943, beta[1]=    0.03214196
12: th=    10.14679104, dev=135035.36618937, beta[1]=    0.03214196
13: th=    10.14696051, dev=135035.36618935, beta[1]=    0.03214196


┌ Warning: RCall.jl: theta.ml: iter 0 'theta = 1.006550'
│ theta.ml: iter1 theta =1.50045
│ theta.ml: iter2 theta =2.17571
│ theta.ml: iter3 theta =3.08018
│ theta.ml: iter4 theta =4.26851
│ theta.ml: iter5 theta =5.79776
│ theta.ml: iter6 theta =7.71595
│ theta.ml: iter7 theta =10.037
│ theta.ml: iter8 theta =12.6918
│ theta.ml: iter9 theta =15.4473
│ theta.ml: iter10 theta =17.8272
│ theta.ml: iter11 theta =19.2399
│ theta.ml: iter12 theta =19.6259
│ theta.ml: iter13 theta =19.6492
│ theta.ml: iter14 theta =19.6493
└ @ RCall /Users/sarahji/.julia/packages/RCall/6kphM/src/io.jl:172


RObject{RealSxp}
   user  system elapsed 
 74.146   1.500  75.774 


In [13]:
### Show estimates
R"""
    m.nb
"""

RObject{S4Sxp}
Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: Negative Binomial(10.147)  ( log )
Formula: Y ~ 1 + X2 + X3 + (1 | group)
   Data: df
      AIC       BIC    logLik  deviance  df.resid 
135045.37 135089.47 -67517.68 135035.37     49995 
Random effects:
 Groups Name        Std.Dev.
 group  (Intercept) 0.09233 
Number of obs: 50000, groups:  group, 10000
Fixed Effects:
(Intercept)           X2           X3  
    0.03214      0.10575      0.02603  


In [14]:
# Show estimated r from lme4: glmer.nb
R"""
    getME(m.nb, "glmer.nb.theta")
"""

RObject{RealSxp}
[1] 10.14696


In [15]:
### Show confidence intervals from lme4: glmer.nb
R"""
    confint(m.nb)
"""

┌ Warning: RCall.jl: Computing profile confidence intervals ...
└ @ RCall /Users/sarahji/.julia/packages/RCall/6kphM/src/io.jl:172


RObject{RealSxp}
                 2.5 %     97.5 %
.sig01      0.04865489 0.12192888
(Intercept) 0.02226002 0.04192188
X2          0.09662966 0.11486875
X3          0.01700341 0.03505898


In [16]:
# time confidence intervals
R"""
    ptm <- proc.time()
    confint(m.nb)
    # Stop the clock
    proc.time() - ptm
"""

┌ Warning: RCall.jl: Computing profile confidence intervals ...
└ @ RCall /Users/sarahji/.julia/packages/RCall/6kphM/src/io.jl:172


RObject{RealSxp}
   user  system elapsed 
154.353   3.575 158.034 


# Using glmmTMB package in R

In [17]:
R"""
    library("glmmTMB")
    ptm <- proc.time()
    m.glmmtmb_nb <- glmmTMB(Y ~ 1 + X2 + X3 + (1|group), data = df, family=nbinom2)
    # Stop the clock
    proc.time() - ptm
"""

┌ Warning: RCall.jl: Warning: package ‘glmmTMB’ was built under R version 4.0.2
│ Warning in checkDepPackageVersion(dep_pkg = "TMB") :
│   Package version inconsistency detected.
│ glmmTMB was built with TMB version 1.7.21
│ Current TMB version is 1.7.22
│ Please re-install glmmTMB from source or restore original ‘TMB’ package (see '?reinstalling' for more information)
└ @ RCall /Users/sarahji/.julia/packages/RCall/6kphM/src/io.jl:172


RObject{RealSxp}
   user  system elapsed 
 98.204   0.670  98.944 


In [18]:
### Show estimates
R"""
    m.glmmtmb_nb
"""

RObject{VecSxp}
Formula:          Y ~ 1 + X2 + X3 + (1 | group)
Data: df
      AIC       BIC    logLik  df.resid 
135045.71 135089.81 -67517.86     49995 
Random-effects (co)variances:

Conditional model:
 Groups Name        Std.Dev.
 group  (Intercept) 0.08967 

Number of obs: 50000 / Conditional model: group, 10000

Dispersion parameter for nbinom2 family (): 10.1 

Fixed Effects:

Conditional model:
(Intercept)           X2           X3  
    0.03276      0.10579      0.02604  


In [19]:
# Show estimated r from glmmTMB
R"""
    sigma(m.glmmtmb_nb)
"""

RObject{RealSxp}
[1] 10.10115


In [20]:
### Show confidence intervals from glmmTMB
R"""
    confint(m.glmmtmb_nb, full = TRUE)
"""

RObject{RealSxp}
                                    2.5 %      97.5 %    Estimate
cond.(Intercept)               0.02290637  0.04261165  0.03275901
cond.X2                        0.09666585  0.11491162  0.10578874
cond.X3                        0.01701310  0.03507562  0.02604436
sigma                          8.64035445 11.80892546 10.10115348
group.cond.Std.Dev.(Intercept) 0.05832825  0.13786618  0.08967437


In [21]:
# time confidence intervals
R"""
    ptm <- proc.time()
    confint(m.glmmtmb_nb, full = TRUE)
    # Stop the clock
    proc.time() - ptm
"""

RObject{RealSxp}
   user  system elapsed 
  0.463   0.008   0.472 


# Using GLMMadaptive

In [22]:
R"""
    library("GLMMadaptive")
    ptm <- proc.time()
    fm <- mixed_model(fixed = Y ~ 1 + X2 + X3, random = ~ 1 | group, data = df,
                  family = negative.binomial(), nAGQ = 25)
    # Stop the clock
    proc.time() - ptm
"""

┌ Warning: RCall.jl: Warning: package ‘GLMMadaptive’ was built under R version 4.0.5
│ 
│ Attaching package: ‘GLMMadaptive’
│ 
│ The following object is masked from ‘package:lme4’:
│ 
│     negative.binomial
│ 
└ @ RCall /Users/sarahji/.julia/packages/RCall/6kphM/src/io.jl:172


RObject{RealSxp}
   user  system elapsed 
 75.012   9.420  84.471 


In [23]:
### Show estimates
R"""
    summary(fm)
"""

RObject{VecSxp}

Call:
mixed_model(fixed = Y ~ 1 + X2 + X3, random = ~1 | group, data = df, 
    family = negative.binomial(), nAGQ = 25)

Data Descriptives:
Number of Observations: 50000
Number of Groups: 10000 

Model:
 family: negative binomial
 link: log 

Fit statistics:
   log.Lik      AIC      BIC
 -67518.14 135046.3 135082.3

Random effects covariance matrix:
                StdDev
(Intercept) 0.09531734

Fixed effects:
            Estimate Std.Err z-value p-value
(Intercept)   0.0323  0.0050  6.4869 < 1e-04
X2            0.1058  0.0047 22.7081 < 1e-04
X3            0.0260  0.0046  5.6470 < 1e-04

log(dispersion) parameter:
  Estimate Std.Err
    2.3022   0.076

Integration:
method: adaptive Gauss-Hermite quadrature rule
quadrature points: 25

Optimization:
method: hybrid EM and quasi-Newton
converged: TRUE 


In [24]:
### Show estimate of r from GLMMadaptive
exp(2.3022)

9.996149811447443

In [25]:
### Show confidence intervals of fixed effects from GLMMadaptive
R"""
    confint(fm)
"""

RObject{RealSxp}
                 2.5 %   Estimate     97.5 %
(Intercept) 0.02251628 0.03226494 0.04201360
X2          0.09667221 0.10580433 0.11493646
X3          0.01700637 0.02604673 0.03508708


In [26]:
### Show confidence interval of variance component from GLMMadaptive
R"""
    confint(fm, "var-cov")
"""

RObject{RealSxp}
                      2.5 %    Estimate     97.5 %
var.(Intercept) 0.004617391 0.009085395 0.01787685


In [27]:
### Show confidence interval of r from GLMMadaptive
R"""
    confint(fm, "extra")
"""

RObject{RealSxp}
         2.5 % Estimate   97.5 %
phi_1 8.611931 9.995654 11.60171


In [28]:
# time confidence intervals
R"""
    ptm <- proc.time()
    confint(fm)
    confint(fm, "var-cov")
    confint(fm, "extra")
    # Stop the clock
    proc.time() - ptm
"""

RObject{RealSxp}
   user  system elapsed 
  0.001   0.000   0.000 
